In [48]:
import pandas as pd
import re

In [133]:
data = pd.read_csv('D:\\博士班\\Aesop\\Docly\\MIMIC\\20250611_extract_title2.csv')
data.iloc[25:30,:]

,subject_id,hadm_id,transfertime,prev_service,curr_service,admittime,dischtime,deathtime,admission_type,admit_provider_id,...,anchor_age,anchor_year,anchor_year_group,dod,note_id,note_type,note_seq,charttime,storetime,text
25,11464524,25770700,2146-10-04 12:17:54,NaN,ORTHO,2146-10-04 12:17:00,2146-10-05 15:40:00,NaN,EW EMER.,P13ACE,...,18,2146,2017 - 2019,NaN,11464524-DS-16,DS,16.0,2146-10-05 00:00:00,2146-10-05 15:22:00,\nName: ___ Unit No: ___\n \n...
26,11469935,25962795,2183-10-16 03:23:15,NaN,ORTHO,2183-10-16 03:22:00,2183-10-16 16:01:00,NaN,EW EMER.,P13ACE,...,18,2183,2011 - 2013,NaN,11469935-DS-21,DS,21.0,2183-10-16 00:00:00,2183-10-16 14:29:00,\nName: ___ Unit No: ___\...
27,11765403,22379004,2186-07-27 19:02:21,NaN,ORTHO,2186-07-27 19:01:00,2186-07-30 14:58:00,NaN,OBSERVATION ADMIT,P13ACE,...,18,2186,2017 - 2019,NaN,11765403-DS-8,DS,8.0,2186-07-30 00:00:00,2186-07-30 15:22:00,\nName: ___ Unit No: ___\n \...
28,11947568,22237885,2116-09-14 05:10:18,NaN,ORTHO,2116-09-14 10:30:00,2116-09-15 13:25:00,NaN,ELECTIVE,P57BOT,...,18,2116,2011 - 2013,NaN,11947568-DS-18,DS,18.0,2116-09-15 00:00:00,2116-09-15 06:15:00,\nName: ___ Unit No: __...
29,11947568,24193887,2117-01-11 03:17:24,NaN,ORTHO,2117-01-11 12:45:00,2117-01-18 10:16:00,NaN,DIRECT EMER.,P574ZF,...,18,2116,2011 - 2013,NaN,11947568-DS-19,DS,19.0,2117-01-18 00:00:00,2117-01-18 08:14:00,\nName: ___ Unit No: __...


In [134]:
len(data)

26991

In [135]:
## N = 1501 有串到Discharge note
data1 = data[data['text'].notna()].reset_index(drop=True)
len(data1)

20058

In [136]:
def title_extract(text):
    line = text.splitlines()
    # title_list = [i.strip() for i in line if (i.strip().endswith(':') or i.strip().endswith('：')) and re.match(r'^[^\s].*[:：]\s*$', i)]
    title_list = [i.strip() for i in line if re.match(r'^[^\s].*[:：]\s*$', i)]
    return title_list

In [142]:
data1['text_title'] = data1['text'].apply(title_extract)
for i in data1['text_title']:
    print(i)
    break

['Allergies:', 'Chief Complaint:', 'Major Surgical or Invasive Procedure:', 'History of Present Illness:', 'Past Medical History:', 'Social History:', 'Family History:', 'Physical Exam:', 'On Admission:', 'Right upper extremity:', 'Left upper extremity:', 'On Discharge:', 'Pertinent Results:', 'Brief Hospital Course:', 'Medications on Admission:', 'Discharge Medications:', 'Discharge Disposition:', 'Discharge Diagnosis:', 'Discharge Condition:', 'Discharge Instructions:', 'MEDICATIONS:', 'WOUND CARE:', 'ACTIVITY AND WEIGHT BEARING:', 'Followup Instructions:']


In [143]:
## 最後拿出一個distinct的title列表
all_title = []
for i in data1['text_title']:
    all_title.extend(i)
# print(all_title)
    # break

In [144]:
distinct_title = list(set(all_title))

In [145]:
print(len(all_title))
print(len(distinct_title))

464394
10615


In [146]:
distinct_title

['Head CT      Pos () Neg () N/A (x) Findings:',
 'Assessment/Plan:',
 'WOUND CULTURE (Final ___:',
 '3) OPAT Antimicrobial Regimen and Projected Duration:',
 '___: XR femur:',
 'pelvis was revealing for:',
 '# Pain control:',
 'SEIZURES, BLACKOUTS, DETOX TREATMENT, I.V. USAGE):',
 'TRANSTHORACIC ECHOCARDIOGRAM ___:',
 'K-4.5 Cl-105 HCO3-24 AnGap-___RIEF CLINICAL SUMMARY:',
 'UNILAT UP EXT VEINS US RIGHT showed:',
 '___ Imaging RENAL TRANSPLANT US:',
 '8. Hypothyroidism:',
 '8. Lamotrigine 25 mg Tablet Sig: Two (2) Tablet PO twice a day:',
 'MICRO LABS:',
 '**Follow Up Issues:',
 'Vitals in PACU:',
 'infarcts, chronic changes.  MRI/MRA IMPRESSION:',
 '6. metformin 500 mg Tablet Sig: One (1) Tablet PO DAILY (Daily):',
 'Follow Up:',
 'morning. The following CSF studies should be sent:',
 'R ankle Xrays ___:',
 'GRAM STAIN (Final ___:',
 'Exercise:',
 '17. Alum-Mag Hydroxide-Simeth 200-200-20 mg/5 mL Suspension Sig:',
 'VS ___ 0338 Temp: 101.8 PO BP: 130/69 HR: 91 RR: 18 O2 sat:',
 'Imag

In [147]:
title_dash = []
title_no_dash = []
for i in distinct_title:
    if i.startswith('___'):
        title_dash.append(i)
    else:
        title_no_dash.append(i)

In [148]:
df = pd.DataFrame({'title': distinct_title})
title_dash = pd.DataFrame({'title':title_dash})
title_no_dash = pd.DataFrame({'title':title_no_dash})


In [150]:
len(title_no_dash)

9578

In [151]:
with pd.ExcelWriter('D:/博士班/Aesop/Docly/MIMIC/title_20250611(2).xlsx', engine='openpyxl') as writer:
    data1.to_excel(writer, sheet_name='All data', index=False)
    df.to_excel(writer, sheet_name='Distinct titles (10615)', index=False)
    title_dash.to_excel(writer, sheet_name='title_dash (1037)', index=False)
    title_no_dash.to_excel(writer, sheet_name='title_no_dash (9578)', index=False)

### 附錄

In [115]:
## 把text內容拿出來，冒號結尾，且是該行第一個開頭，前面不能有空白
for text in data1['text']:
    line = text.splitlines()    
    print(line)
    title_list = [i.strip() for i in line if (i.endswith(':') or i.endswith('：')) and re.match(r'^[^\s].*[:：]\s*$', i)]
    print(title_list)
    break

[' ', 'Name:  ___                     Unit No:   ___', ' ', 'Admission Date:  ___              Discharge Date:   ___', ' ', 'Date of Birth:  ___             Sex:   M', ' ', 'Service: ORTHOPAEDICS', ' ', 'Allergies: ', 'No Known Allergies / Adverse Drug Reactions', ' ', 'Attending: ___.', ' ', 'Chief Complaint:', 'Left Arm Pain', ' ', 'Major Surgical or Invasive Procedure:', 'Open reduction and internal fixation of left both bones forearm ', 'fracture (___)', '', ' ', 'History of Present Illness:', '___, ___, presents after a snowboarding accident at ___ ', 'in which he sustained a midshaft radius and ulna fx, seen by XR ', 'at the mountain, and splinted at the mountain.  Pt reports he ', 'was wearing a helmet, had no LOC or head strike, and denies any ', 'other injuries. ', '', ' ', 'Past Medical History:', 'None', ' ', 'Social History:', '___', 'Family History:', 'Non-contributory.', ' ', 'Physical Exam:', 'On Admission:', 'Gen: NAD', 'Vitals: 98.8 71 130/61 16 98% ', 'CV RRR', 'Pulm:

In [116]:
## 把text內容拿出來，冒號結尾，且是該行第一個開頭，前面不能有空白
# 這個才對
for text in data1['text']:
    line = text.splitlines()    
    print(line)
    title_list = [i.strip() for i in line if re.match(r'^[^\s].*[:：]\s*$', i)]
    print(title_list)
    break

[' ', 'Name:  ___                     Unit No:   ___', ' ', 'Admission Date:  ___              Discharge Date:   ___', ' ', 'Date of Birth:  ___             Sex:   M', ' ', 'Service: ORTHOPAEDICS', ' ', 'Allergies: ', 'No Known Allergies / Adverse Drug Reactions', ' ', 'Attending: ___.', ' ', 'Chief Complaint:', 'Left Arm Pain', ' ', 'Major Surgical or Invasive Procedure:', 'Open reduction and internal fixation of left both bones forearm ', 'fracture (___)', '', ' ', 'History of Present Illness:', '___, ___, presents after a snowboarding accident at ___ ', 'in which he sustained a midshaft radius and ulna fx, seen by XR ', 'at the mountain, and splinted at the mountain.  Pt reports he ', 'was wearing a helmet, had no LOC or head strike, and denies any ', 'other injuries. ', '', ' ', 'Past Medical History:', 'None', ' ', 'Social History:', '___', 'Family History:', 'Non-contributory.', ' ', 'Physical Exam:', 'On Admission:', 'Gen: NAD', 'Vitals: 98.8 71 130/61 16 98% ', 'CV RRR', 'Pulm: